# SAA2 Processing

* Mission Insight csv file did not have a gcs_location parameter; added manually. Ensure it's recorded for future flights.
* The dataflash file is the same for Flights 1-3. Fixed by separating out into three files. 

# To Do
Add objective path to "SAA2.waypoints" files for each one. 

In [3]:
import os
import sys
import json
import shutil
PATH = "/home/samuel/SpiderOak Hive/ACUASI/Code_Repos/nasa-tcl3-data-generator"
sys.path.append(PATH)
from tcl3parsers import saa2, flight_data

rootdir = "/home/samuel/SpiderOak Hive/ACUASI/NASA TO6/Mission Data/SAA2"

days = []
aircraft = []
flights = []

UASORG = "ACUASI"

FLAG = 0

if os.path.exists(rootdir + "/deliverables"):
    shutil.rmtree(rootdir + "/deliverables")           

for dirpath, dirnames, filenames in os.walk(rootdir, topdown=False):
    mi = ""
    dataflash = ""
    saa_ti = ""
    saa_td = ""
    if filenames:
        print("Processing directory: " + dirpath)
        outdir = rootdir + "/deliverables/" + dirpath.replace(rootdir, "")
        for file in filenames:
            if file.endswith(".log"):
                dataflash = os.path.join(dirpath, file)
                ts = file.strip(".log")
                date = ts.split(" ")[0].replace("-", "")
                time = ts.split(" ")[1].replace("-", "")[0:4]
                date2 = ts.split(" ")[0]
            if file.endswith("mission_insight.csv"):
                mi = os.path.join(dirpath, file)
            if file.endswith("TI.csv"):
                saa_ti = os.path.join(dirpath, file)
            if file.endswith("TD.csv"):
                saa_td = os.path.join(dirpath, file)
            if file == "SAA2.waypoints":
                src = os.path.join(dirpath, file)
        fd = "{}/{}-{}-{}-FLIGHT_DATA.json".format(outdir, UASORG, date, time)
        fd_name = os.path.join(rootdir, fd)
        s = "{}/{}-{}-{}-SAA2.json".format(outdir, UASORG, date, time)
        s_name = os.path.join(rootdir, s)
        dst = "{}/SAA2.waypoints".format(outdir)
        dst = os.path.join(rootdir, dst)
        os.makedirs(os.path.dirname(fd_name), exist_ok=True)
        os.makedirs(os.path.dirname(s_name), exist_ok=True)
        os.makedirs(os.path.dirname(dst), exist_ok=True)
        shutil.copyfile(src, dst)
        flight_data.generate(mi, dataflash, fd_name)
        saa2.generate(mi, saa_ti, saa_td, s_name)
            
print("Files generated.")

# Add `targetWaypoint*` variables to UAS State object in FLIGHT_DATA.json for first set of SAA2 flights
# where waypoints were not used during flight but were calculated after the fact.
for dirpath, dirnames, filenames in os.walk(rootdir + "/deliverables"):
    if filenames:
        for file in filenames:
            if file.endswith("FLIGHT_DATA.json"):
                print("Processing {}".format(file))
                with open(os.path.join(dirpath, file), "r") as fd_file:
                    fd_json = json.load(fd_file)
                    for item in fd_json["uasState"]:
                        try:
                            if item["targetWaypointLat_deg"]:
                                FLAG = 1
                                print("Skipping {}".format(file))
                        except KeyError:
                            pass
                if not FLAG:
                    with open(os.path.join(dirpath, "SAA2.waypoints"), "r") as wpts:
                        for line in wpts:
                            values = line.split("\t")
                            if values[0] == "1":
                                lat = float(values[8])
                                lon = float(values[9])
                                alt = float(values[10]) * 3.28
                    with open(os.path.join(dirpath, file), "w") as fd_file:
                        time = fd_json["auxiliaryUASOperation"]["takeOffTime"]
                        waypoint_lat = {"ts": time, "sensor": "targetWaypointLat_deg", "value": lat}
                        waypoint_lon = {"ts": time, "sensor": "targetWaypointLon_deg", "value": lon}
                        waypoint_alt = {"ts": time, "sensor": "targetWaypointAlt_ft", "value": alt}
                        fd_json["uasState"].insert(0, waypoint_lat)
                        fd_json["uasState"].insert(0, waypoint_lon)
                        fd_json["uasState"].insert(0, waypoint_alt)
                        json.dump(fd_json, fd_file, indent=4)
print("Complete!")

Processing directory: /home/samuel/SpiderOak Hive/ACUASI/NASA TO6/Mission Data/SAA2/2018-04-23/N970UA/Flight 5
Processing directory: /home/samuel/SpiderOak Hive/ACUASI/NASA TO6/Mission Data/SAA2/2018-04-23/N970UA/Flight 2
Processing directory: /home/samuel/SpiderOak Hive/ACUASI/NASA TO6/Mission Data/SAA2/2018-04-23/N970UA/Flight 3
Processing directory: /home/samuel/SpiderOak Hive/ACUASI/NASA TO6/Mission Data/SAA2/2018-04-23/N970UA/Flight 1
Processing directory: /home/samuel/SpiderOak Hive/ACUASI/NASA TO6/Mission Data/SAA2/2018-04-23/N970UA/Flight 4
Processing directory: /home/samuel/SpiderOak Hive/ACUASI/NASA TO6/Mission Data/SAA2/2018-05-10/N970UA/Flight 2
Processing directory: /home/samuel/SpiderOak Hive/ACUASI/NASA TO6/Mission Data/SAA2/2018-05-10/N970UA/Flight 1
Files generated.
Processing ACUASI-20180423-1422-FLIGHT_DATA.json
Processing ACUASI-20180423-1313-FLIGHT_DATA.json
Processing ACUASI-20180423-1341-FLIGHT_DATA.json
Processing ACUASI-20180423-1301-FLIGHT_DATA.json
Processing